# Strict and Lax Type Coercion

As we saw earlier, Pydantic performs a lax type coercion.

In [1]:
from pydantic import BaseModel, ConfigDict, ValidationError

In [2]:
class Model(BaseModel):
    field_1: str
    field_2: float
    field_3: list
    field_4: tuple

Let's see how lax coercion handles some data:

In [3]:
try:
    Model(field_1=100, field_2=1, field_3=(1, 2, 3), field_4=[1, 2, 3])
except ValidationError as ex:
    print(ex)

1 validation error for Model
field_1
  Input should be a valid string [type=string_type, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type


As you can see, with lax coercion, Pydantic was happy with coercing a tuple into a list, a list into a tuple, and an integer into a float.

So, if we just fix that one field, let's see what we get:

In [4]:
Model(field_1="abc", field_2=1, field_3=(1, 2, 3), field_4=[1, 2, 3])

Model(field_1='abc', field_2=1.0, field_3=[1, 2, 3], field_4=(1, 2, 3))

As you can see the list and tuples were coerced just fine, as was the integer to a float.

This may not be the behavior we want. We can of course write custom validators if we need specialized type validation beyond what Pydantic offers us. We could also set lax (or strict conversion) on the model in general (or, as we'll see laterm, we could also set it on a field by field basis.)

For now, we're going to set the model, overall, to be in strict coercion mode.

This is done via `model_config` as well.

In [5]:
class Model(BaseModel):
    model_config = ConfigDict(strict=True)  # default is False
    
    field_1: str
    field_2: float
    field_3: list
    field_4: tuple

And now, if we try our sample example again:

In [6]:
try:
    Model(field_1=100, field_2=1, field_3=(1, 2, 3), field_4=[1, 2, 3])
except ValidationError as ex:
    print(ex)

2 validation errors for Model
field_1
  Input should be a valid string [type=string_type, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type
field_4
  Input should be a valid tuple [type=tuple_type, input_value=[1, 2, 3], input_type=list]
    For further information visit https://errors.pydantic.dev/2.5/v/tuple_type


You'll notice that the validation exceptions are slightly different.

We have one extra one, coercing of a list into a tuple was not possible. But coercing a tuple into a list is still acceptable.

Again, beware of assuming that no type coercion will take place when you set the mode to strict. Some still will, not as much as lax mode, but still some.

For specific information on what lax and strict coercion means, we can look at the Pydantic docs [here](https://docs.pydantic.dev/latest/concepts/conversion_table/)

Look at the list type, and see if coercing a tuple to a list is supported in strict mode. What about lax mode?

Now do the same for coercing to tuples - see what the documentation say is supported for both lax and strict mode.

So far, we have only really looked at type coercions for Python data types.

But, when deserializing JSON data, the behavior is slightly different.

JSON is quite simple, it does not have many different types like Python does.

Basically JSON understands these data types:
- **boolean**: `true` or `false` (Note the lower case)
- **number** (float or integer does not actually matter)
- **string** (enclosed by double quotes)
- a **null** value (represented by the characters `null`, no surrounding quotes)
- an **array** (square brackets, comma separated values of any JSON type, including arrays or objects)
- an **object** (basically a dictionary, delimited by curly braces, with key: value pairs, where keys must be strings, and values can be any JSON type, including an object itself)

For example:

In [7]:
json_data = '''
{
    "field_1": true,
    "field_2": 10.5,
    "field_3": 10,
    "field_4": null,
    "field_5": [1, 2, 3],
    "field_6": {
        "a": 1,
        "b": 2,
        "c": [3, 4, 5]
    },
    "field_7": [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]
    ]
}
'''

Python supports json deserialization (from JSON to dictionary) natively with the `json` module's `loads()` function.

In [8]:
import json
from pprint import pprint

data = json.loads(json_data)
pprint(data)

{'field_1': True,
 'field_2': 10.5,
 'field_3': 10,
 'field_4': None,
 'field_5': [1, 2, 3],
 'field_6': {'a': 1, 'b': 2, 'c': [3, 4, 5]},
 'field_7': [[1, 0, 0], [0, 1, 0], [0, 0, 1]]}


You'll notice that Python had to make certain assumptions about the data types - it also does it's own form of type coercion.

But the coercion Python does may not be exactly what we want.

For example, notice that all those arrays in the JSON were coerced into lists - maybe we want tuples.

That value `10` was coerced into an `int` - maybe we actually want it to be a float.

Also, the objects (dictionaries), were coerced into Python dictionaries - maybe we want them to be deserialized into a Pydantic model instead of a dictionary! (We'll see nested Pydantic models later in the course).

In [9]:
type(data['field_3'])

int

So, we could turn this into a Pydantic model instead, and get the specific types we want.

Remember that with Python types the coercion was unable to coerce a list into a tuple.

However, when deseriaizing with Pydantic this is not the case. If you look at the documentation I linked above, you'll see that JSON arrays for example can be coerced into lists, tuples, sets, and a few more. And again, there will be slightly stricter coercion rules when using strict coercion mode.

Let's create a model for this JSON data:

In [10]:
json_data = '''
{
    "field_1": true,
    "field_2": 10,
    "field_3": 1,
    "field_4": null,
    "field_5": [1, 2, 3],
    "field_6": ["a", "b", "c"],
    "field_7": {"a": 1, "b": 2}
}
'''

We want the following types for the fields:
- `field_1`: bool
- `field_2`: float
- `field_3`: int
- `field_4`: nullable string
- `field_5`: tuple of integers
- `field_6`: set of strings
- `field_7`: dictionary

In [11]:
class Model(BaseModel):
    field_1: bool
    field_2: float
    field_3: int
    field_4: str | None
    field_5: tuple[int, ...]
    field_6: set[str]
    field_7: dict

We can even use strict mode if we prefer:

In [12]:
class Model(BaseModel):
    model_config = ConfigDict(strict=True)
    
    field_1: bool
    field_2: float
    field_3: int
    field_4: str | None
    field_5: tuple[int, ...]
    field_6: set[str]
    field_7: dict

In [13]:
data = Model.model_validate_json(json_data)

And now, not only have we deserialized the JSON data, but we have the corrcet types, and we validated the data (like the fact that we only accept a tuple of integers, or values that can be coerced to integers).

In [14]:
data

Model(field_1=True, field_2=10.0, field_3=1, field_4=None, field_5=(1, 2, 3), field_6={'a', 'b', 'c'}, field_7={'a': 1, 'b': 2})

In [15]:
type(data.field_5)

tuple

In [16]:
type(data.field_6)

set

In [17]:
type(data.field_7)

dict

If we try to load a JSON object that does not conform to our model schema, for example a float in the tuple, or an integer in the set:

In [18]:
json_data = '''
{
    "field_1": true,
    "field_2": 10,
    "field_3": 1,
    "field_4": null,
    "field_5": [1, 2, 3.5],
    "field_6": ["a", "b", 100],
    "field_7": {"a": 1, "b": 2}
}
'''

try:
    Model.model_validate_json(json_data)
except ValidationError as ex:
    print(ex)
    

2 validation errors for Model
field_5.2
  Input should be a valid integer [type=int_type, input_value=3.5, input_type=float]
    For further information visit https://errors.pydantic.dev/2.5/v/int_type
field_6.2
  Input should be a valid string [type=string_type, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type
